In [1]:
%matplotlib inline


# 3D scattering quantum chemistry regression

Description:
This example trains a classifier combined with a scattering transform to
regress molecular atomization energies on the QM7 dataset. Here, we use full
charges, valence charges and core charges. A linear regression is deployed.

Remarks:
The linear regression of the QM7 energies with the given values gives MAE
2.75, RMSE 4.18 (kcal.mol-1)

Reference:
https://arxiv.org/abs/1805.00571


## Preliminaries

First, we import NumPy, PyTorch, and some utility modules.



In [2]:
import numpy as np
import torch
import time
import os

We will use scikit-learn to construct a linear model, so we import the
necessary modules. In addition, we need to compute distance matrices when
normalizing our input features, so we import `pdist` from `scipy.spatial`.



In [3]:
from sklearn import (linear_model, model_selection, preprocessing,
                     pipeline)
from scipy.spatial.distance import pdist

We then import the necessary functionality from Kymatio. First, we need the
PyTorch frontend of the 3D solid harmonic cattering transform.



In [4]:
from kymatio.torch import HarmonicScattering3D

The 3D transform doesn't compute the zeroth-order coefficients, so we need
to import `compute_integrals` to do this manually.



In [5]:
from kymatio.scattering3d.backend.torch_backend \
    import TorchBackend3D

To generate the input 3D maps, we need to calculate sums of Gaussians, so we
import the function `generate_weighted_sum_of_gaussians`.



In [6]:
from kymatio.scattering3d.utils \
    import generate_weighted_sum_of_gaussians

Finally, we import the utility functions that let us access the QM7 dataset
and the cache directories to store our results.



In [7]:
from kymatio.datasets import fetch_qm7
from kymatio.caching import get_cache_dir

## Data preparation

Fetch the QM7 database and extract the atomic positions and nuclear charges
of each molecule. This dataset contains 7165 organic molecules with up to
seven non-hydrogen atoms, whose energies were computed using density
functional theory.



In [8]:
import sys
sys.path.append(os.path.abspath(".."))
from utils_project import generate_csv,create_dataframe_from_xyz_files,create_X_y_from_dataframe


csv_path = "../../data/energies/train.csv"
path_data = "../../data/atoms/train"
df_train=create_dataframe_from_xyz_files(path_data,csv_path)
X=df_train[['positions', 'energy', 'charges']]

qm7 = X.to_dict("list")

#qm7 = fetch_qm7(align=True)
pos = np.array(qm7['positions'])
full_charges = np.array(qm7['charges'])

n_molecules = pos.shape[0]

In [9]:
print(X.head)

<bound method NDFrame.head of                                               positions     energy  \
0     [[0.356436, -3.423619, -0.385554], [0.674042, ... -90.107880   
1     [[-0.667657, -1.712084, 1.740414], [-1.482199,... -69.210846   
2     [[-2.276677, 1.630799, -0.541086], [-1.370793,... -64.983899   
3     [[-0.040127, -0.466726, 2.319016], [-0.302443,... -76.559740   
4     [[-0.372357, 0.364578, -1.469862], [-0.950262,... -62.270961   
...                                                 ...        ...   
6586  [[-0.840208, 2.415012, 0.629657], [0.327734, 1... -84.253987   
6587  [[-2.076656, -0.640822, -0.564519], [-0.647519... -64.082905   
6588  [[0.959341, 0.33756, 2.251651], [0.014951, -0.... -74.736455   
6589  [[-0.378348, -2.141699, 2.069565], [-1.148777,... -84.180692   
6590  [[-2.173342, 0.950873, -0.201567], [-0.709742,... -62.286505   

                                                charges  
0     [6, 6, 6, 6, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 1, ...  
1     [6, 6, 

In [10]:
for clé, valeur in qm7.items():
    print(f"Clé: {clé}, Haut de la liste: {valeur[:30]}")

print(qm7.keys())

Clé: positions, Haut de la liste: [array([[ 0.356436, -3.423619, -0.385554],
       [ 0.674042, -1.997613, -0.81716 ],
       [-0.160963, -0.953233, -0.067053],
       [ 0.15581 ,  0.481542, -0.509487],
       [-0.711321,  1.516257,  0.195147],
       [-0.257499,  2.171619,  1.453771],
       [-0.28779 ,  2.919863,  0.195085],
       [ 0.951293, -4.140487, -0.947535],
       [-0.693579, -3.657572, -0.54993 ],
       [ 0.567462, -3.568831,  0.672098],
       [ 0.501194, -1.89482 , -1.893711],
       [ 1.738769, -1.795955, -0.65804 ],
       [ 0.010933, -1.051982,  1.010204],
       [-1.225808, -1.156433, -0.22521 ],
       [ 0.012401,  0.563854, -1.594183],
       [ 1.213274,  0.70068 , -0.323745],
       [-1.790995,  1.380711,  0.058204],
       [ 0.723998,  1.91488 ,  1.857359],
       [-0.989227,  2.485891,  2.199616],
       [ 0.655782,  3.0113  , -0.188705],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ],
       [ 0.      ,  0.      ,  0.      ]]

From the nuclear charges, we compute the number of valence electrons, which
we store as the valence charge of that atom.



In [11]:
mask = full_charges <= 2
valence_charges = full_charges * mask

mask = np.logical_and(full_charges > 2, full_charges <= 10)
valence_charges += (full_charges - 2) * mask

mask = np.logical_and(full_charges > 10, full_charges <= 18)
valence_charges += (full_charges - 10) * mask

We then normalize the positions of the atoms. Specifically, the positions
are rescaled such that two Gaussians of width `sigma` placed at those
positions overlap with amplitude less than `overlapping_precision`.



In [12]:
overlapping_precision = 1e-1
sigma = 2.0
min_dist = np.inf

for i in range(n_molecules):
    n_atoms = np.sum(full_charges[i] != 0)
    pos_i = pos[i, :n_atoms, :]
    min_dist = min(min_dist, pdist(pos_i).min())

delta = sigma * np.sqrt(-8 * np.log(overlapping_precision))
pos = pos * delta / min_dist

## Scattering Transform
Given the rescaled positions and charges, we are now ready to compute the
density maps by placing Gaussians at the different positions weighted by the
appropriate charge. These are fed into the 3D solid harmonic scattering
transform to obtain features that are used to regress the energies. In
order to do this, we must first define a grid.



In [13]:
M, N, O = 32, 32, 32 #192, 128, 96
grille = "32-32-32"
grid = np.mgrid[-M//2:-M//2+M, -N//2:-N//2+N, -O//2:-O//2+O]
grid = np.fft.ifftshift(grid)

We then define the scattering transform using the `HarmonicScattering3D`
class.



In [14]:
J = 2
L = 3
integral_powers = [0.5, 1.0, 2.0, 3.0]

scattering = HarmonicScattering3D(J=J, shape=(M, N, O),
                                  L=L, sigma_0=sigma,
                                  integral_powers=integral_powers)

We then check whether a GPU is available, in which case we transfer our
scattering object there.



In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
scattering.to(device)
print(device)

cuda


The maps computed for each molecule are quite large, so the computation has
to be done by batches. Here we select a small batch size to ensure that we
have enough memory when running on the GPU. Dividing the number of molecules
by the batch size then gives us the number of batches.



In [16]:
batch_size = 8
n_batches = int(np.ceil(n_molecules / batch_size))

We are now ready to compute the scattering transforms. In the following
loop, each batch of molecules is transformed into three maps using Gaussians
centered at the atomic positions, one for the nuclear charges, one for the
valence charges, and one with their difference (called the “core” charges).
For each map, we compute its scattering transform up to order two and store
the results.



In [17]:
order_0, orders_1_and_2 = [], []
print('Computing solid harmonic scattering coefficients of '
      '{} molecules from the QM7 database on {}'.format(
        n_molecules, "GPU" if use_cuda else "CPU"))
print('sigma: {}, L: {}, J: {}, integral powers: {}'.format(
        sigma, L, J, integral_powers))

this_time = None
last_time = None

for i in range(n_batches):
    this_time = time.time()
    if last_time is not None:
        dt = this_time - last_time
        print("Iteration {} ETA: [{:02}:{:02}:{:02}]".format(
            i + 1, int(((n_batches - i - 1) * dt) // 3600),
            int((((n_batches - i - 1) * dt) // 60) % 60),
            int(((n_batches - i - 1) * dt) % 60)))
    else:
        print("Iteration {} ETA: {}".format(i + 1, '-'))
    last_time = this_time
    time.sleep(1)

    # Extract the current batch.
    start = i * batch_size
    end = min(start + batch_size, n_molecules)

    pos_batch = pos[start:end]
    full_batch = full_charges[start:end]
    val_batch = valence_charges[start:end]

    # === Nuclear density ===
    full_density_batch = generate_weighted_sum_of_gaussians(grid,
                                pos_batch, full_batch, sigma)
    full_density_batch = torch.from_numpy(full_density_batch).float().to(device)

    full_order_0 = TorchBackend3D.compute_integrals(full_density_batch, integral_powers)
    full_scattering = scattering(full_density_batch)

    # === Valence density ===
    val_density_batch = generate_weighted_sum_of_gaussians(grid,
                                pos_batch, val_batch, sigma)
    val_density_batch = torch.from_numpy(val_density_batch).float().to(device)

    val_order_0 = TorchBackend3D.compute_integrals(val_density_batch, integral_powers)
    val_scattering = scattering(val_density_batch)

    # === Core density ===
    core_density_batch = full_density_batch - val_density_batch
    core_order_0 = TorchBackend3D.compute_integrals(core_density_batch, integral_powers)
    core_scattering = scattering(core_density_batch)

    # === Stack all coefficients ===
    batch_order_0 = torch.stack((full_order_0, val_order_0, core_order_0), dim=-1).to(device)
    batch_orders_1_and_2 = torch.stack((full_scattering, val_scattering, core_scattering), dim=-1).to(device)

    order_0.append(batch_order_0)
    orders_1_and_2.append(batch_orders_1_and_2)


Computing solid harmonic scattering coefficients of 6591 molecules from the QM7 database on GPU
sigma: 2.0, L: 3, J: 2, integral powers: [0.5, 1.0, 2.0, 3.0]
Iteration 1 ETA: -
Iteration 2 ETA: [00:17:45]
Iteration 3 ETA: [00:16:32]
Iteration 4 ETA: [00:16:22]
Iteration 5 ETA: [00:16:20]
Iteration 6 ETA: [00:16:26]
Iteration 7 ETA: [00:16:19]
Iteration 8 ETA: [00:16:20]
Iteration 9 ETA: [00:16:26]
Iteration 10 ETA: [00:16:23]
Iteration 11 ETA: [00:16:12]
Iteration 12 ETA: [00:16:17]
Iteration 13 ETA: [00:16:15]
Iteration 14 ETA: [00:16:14]
Iteration 15 ETA: [00:16:14]
Iteration 16 ETA: [00:16:06]
Iteration 17 ETA: [00:15:59]
Iteration 18 ETA: [00:16:08]
Iteration 19 ETA: [00:16:06]
Iteration 20 ETA: [00:16:05]
Iteration 21 ETA: [00:16:05]
Iteration 22 ETA: [00:16:07]
Iteration 23 ETA: [00:16:01]
Iteration 24 ETA: [00:15:59]
Iteration 25 ETA: [00:16:02]
Iteration 26 ETA: [00:16:01]
Iteration 27 ETA: [00:16:02]
Iteration 28 ETA: [00:15:57]
Iteration 29 ETA: [00:15:54]
Iteration 30 ETA: [

In [18]:
# Fusionner les résultats en un seul tenseur
order_0_tensor = torch.cat(order_0, dim=0)
orders_1_and_2_tensor = torch.cat(orders_1_and_2, dim=0)

# Sauvegarder les tenseurs sur disque
torch.save({
    'order_0': order_0_tensor,
    'orders_1_and_2': orders_1_and_2_tensor
}, f'../models_scattering/scattering_outputs_{grille}.pt')

In [19]:
# Charger les données
saved_data = torch.load( f'../models_scattering/scattering_outputs_{grille}.pt', map_location=device)
order_0 = saved_data['order_0']
orders_1_and_2 = saved_data['orders_1_and_2']

Concatenate the batch outputs and transfer to NumPy.



In [20]:
#order_0 = torch.cat(order_0, dim=0)
#orders_1_and_2 = torch.cat(orders_1_and_2, dim=0)

order_0 = order_0.cpu().numpy()
orders_1_and_2 = orders_1_and_2.cpu().numpy()

## Regression

To use the scattering coefficients as features in a scikit-learn pipeline,
these must be of shape `(n_samples, n_features)`, so we reshape our arrays
accordingly.



In [21]:
order_0 = order_0.reshape((n_molecules, -1))
orders_1_and_2 = orders_1_and_2.reshape((n_molecules, -1))

Since the above calculation is quite lengthy, we save the results to a cache
for future use.



In [22]:
basename = 'qm7_L_{}_J_{}_sigma_{}_MNO_{}_powers_{}.npy'.format(
        L, J, sigma, (M, N, O), integral_powers)

cache_dir = get_cache_dir("qm7/experiments")

filename = os.path.join(cache_dir, 'order_0_' + basename)
np.save(filename, order_0)

filename = os.path.join(cache_dir, 'orders_1_and_2' + basename)
np.save(filename, orders_1_and_2)

We now concatenate the zeroth-order coefficients with the rest since we want
to use all of them as features.



In [23]:
scattering_coef = np.concatenate([order_0, orders_1_and_2], axis=1)

In [24]:
print(scattering_coef[:10])

[[3.7473785e+02 3.3206824e+02 1.3969110e+02 ... 1.0189753e-09
  2.7360919e-10 4.3759954e-11]
 [3.8754907e+02 3.3159900e+02 1.8637729e+02 ... 6.1500971e-10
  2.2620333e-10 1.4414978e-11]
 [3.9093448e+02 3.4189935e+02 1.5895160e+02 ... 8.4221197e-10
  2.3624075e-10 3.3636708e-11]
 ...
 [4.9320163e+02 4.2605374e+02 2.1924141e+02 ... 5.9445970e-10
  1.6625482e-10 2.4213149e-11]
 [3.7122061e+02 3.1620593e+02 1.7626868e+02 ... 1.0031602e-09
  2.6874500e-10 4.2882926e-11]
 [4.4504340e+02 3.7944678e+02 2.1133316e+02 ... 7.9684981e-10
  2.1648890e-10 3.3559294e-11]]


Fetch the target energies from the QM7 dataset.



In [25]:
target = qm7['energy']

In [26]:
print(target)

[-90.10787994300516, -69.21084563000386, -64.98389905500244, -76.55974048400094, -62.27096080599768, -75.88599541100393, -83.34654316199885, -74.87360933800392, -77.35551558700354, -97.05809972400584, -91.23617604600533, -84.67336226500447, -62.27547180599686, -90.2546750460051, -54.50982520100206, -79.00292158700358, -83.51960426500409, -54.83202420100224, -73.4079231619944, -68.12486255700333, -97.1681477240054, -72.14295680600117, -76.74888251400444, -84.84214626500398, -83.88131526500456, -91.80152904600529, -69.74227866000365, -91.00229704600495, -83.55611816199962, -70.03653483600374, -82.3271800890052, -61.88693298200269, -69.3964646600034, -66.9258294839965, -69.23150263000389, -84.02226826500464, -64.11386195200248, -91.7849910460045, -78.63462858700314, -76.76334448399939, -62.74910895200264, -97.2078147240054, -73.29973216199505, -79.02515458700327, -83.9870172650044, -90.52203504600492, -69.69555263000348, -84.45656326500466, -75.31763030800448, -83.95196826500433, -90.7499

We evaluate the performance of the regression using five-fold
cross-validation. To do so, we first shuffle the molecules, then we store
the resulting indices in `cross_val_folds`.



In [27]:
n_folds = 3

P = np.random.permutation(n_molecules).reshape((n_folds, -1))

cross_val_folds = []

for i_fold in range(n_folds):
    fold = (np.concatenate(P[np.arange(n_folds) != i_fold], axis=0),
            P[i_fold])
    cross_val_folds.append(fold)

Given these folds, we compute the regression error for various settings of
the `alpha` parameter, which controls the amount of regularization applied
to the regression problem (here in the form of a simple ridge regression, or
Tikhonov, regularization). The mean absolute error (MAE) and root mean
square error (RMSE) is output for each value of `alpha`.



In [28]:
import numpy as np
from sklearn import linear_model, preprocessing, pipeline, model_selection
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
import joblib

# Liste des modèles à tester
models = [
    ("Ridge Regression with alpha=0.1", linear_model.Ridge(alpha=0.1)),
    ("Ridge Regression with alpha=1", linear_model.Ridge(alpha=1)),
    ("Ridge Regression with alpha=10", linear_model.Ridge(alpha=10)),
    ("Lasso Regression", linear_model.Lasso()),
    ("ElasticNet Regression", linear_model.ElasticNet()),
    ("Random Forest Regression", RandomForestRegressor()),
    ("Support Vector Regression", SVR()),
    ("XGBoost Regression", XGBRegressor()),
    ("MLP Regressor", MLPRegressor(random_state=1, max_iter=2000, tol=0.1))
]

results = []

for name, model in models:
    scaler = preprocessing.StandardScaler()
    regressor = pipeline.make_pipeline(scaler, model)

    target_prediction = model_selection.cross_val_predict(regressor, X=scattering_coef, y=target, cv=cross_val_folds)

    MAE = np.mean(np.abs(target_prediction - target))
    RMSE = np.sqrt(np.mean((target_prediction - target) ** 2))

    results.append((name, model, MAE, RMSE))

    print('{}: MAE: {}, RMSE: {}'.format(name, MAE, RMSE))

# Trouver le modèle avec le RMSE le plus bas
best_result = min(results, key=lambda x: x[3])
best_model_name, best_model, best_mae, best_rmse = best_result

print(f"Le meilleur modèle est {best_model_name} avec un RMSE de {best_rmse}.")

/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=3.07799e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=3.03455e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=2.9865e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Ridge Regression with alpha=0.1: MAE: 5.297603733768784, RMSE: 6.808881301539874
Ridge Regression with alpha=1: MAE: 5.524875915660383, RMSE: 7.003209869172288
Ridge Regression with alpha=10: MAE: 5.725796323418657, RMSE: 7.277521236612368
Lasso Regression: MAE: 7.282503599244559, RMSE: 9.012440174088763
ElasticNet Regression: MAE: 7.09002098321733, RMSE: 8.814459783805008
Random Forest Regression: MAE: 6.065495942281768, RMSE: 7.760386419169502
Support Vector Regression: MAE: 6.565108287195836, RMSE: 8.377725815026357
XGBoost Regression: MAE: 6.113963669862582, RMSE: 7.891473008321974
MLP Regressor: MAE: 5.961532520792026, RMSE: 7.580658505532594
Le meilleur modèle est Ridge Regression with alpha=0.1 avec un RMSE de 6.808881301539874.


In [29]:
# Entraîner le meilleur modèle sur l'ensemble des données
scaler = preprocessing.StandardScaler()
best_regressor = pipeline.make_pipeline(scaler, best_model)
best_regressor.fit(scattering_coef, target)

# Enregistrer le meilleur modèle
joblib.dump(best_regressor, f'../models_scattering/best_model_{grille}.pkl')

/home/gris/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:213: LinAlgWarning: Ill-conditioned matrix (rcond=1.72429e-08): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


['../models_scattering/best_model_32-32-32.pkl']